# Simple RAG Implementation

Based on [Alfredo Deza's GitHub Repository](https://github.com/alfredodeza/learn-retrieval-augmented-generation).

In this notebook we will build a simple RAG application based on a structured CSV file with wine rating. We will:
* [Load the small dataset](#loading-the-dataset).
* [Encode a column using vector embedding](#Encode-using-vector-embedding).
* [**R**etrieve some of the rows based on a query using semantic similarity](#retrieve-sematically-relevant-data-based-on-users-query).
* [**A**ugment the prompt to the LLM with the retrieved data](#augment-the-prompt-to-the-llm-with-retrieved-data).
* [**G**enerate a reply to the user's query based on the retrieved rows](#generate-reply-to-the-users-query).

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich), and `rich-theme-manager` to make the output more readable, and supress warning messages.

In [3]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names 
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names 
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes/dark.theme  
 light  Light mode theme        themes/light.theme

                                      Theme: dark - themes/dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [5]:
from rich.console import Console

dark = theme_manager.get("dark")
# Create a console with the dark theme
console = Console(theme=dark)


In [6]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Loading the Dataset

Since the data is in a simple, small and structured CSV file, we can load it using Pandas.

In [7]:
import pandas as pd

data = (
    pd
    .read_csv('data/top_rated_wines.csv')
    .query('variety.notna()')
    .reset_index(drop=True)
    .to_dict('records')
)
console.print(data[:2])

[
    {
        'name': '3 Rings Reserve Shiraz 2004',
        'region': 'Barossa Valley, Barossa, South Australia, Australia',
        'variety': 'Red Wine',
        'rating': 96.0,
        'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme 
heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004.
Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank 
maturation of the blend prior to bottling, July 2007. '
    },
    {
        'name': 'Abreu Vineyards Cappella 2007',
        'region': 'Napa Valley, California',
        'variety': 'Red Wine',
        'rating': 96.0,
        'notes': 'Cappella is a proprietary blend of two clones of Cabernet Sauvignon with Cabernet Franc, Petit 
Verdot and Merlot. The gravelly soil at Cappella produces fruit that is very elegant in structure. The resulting 
wine exhibits beautiful purity of fruit with fine grained and lengthy tannins. '
    }
]

## Encode using Vector Embedding

We will use one of the popular open source vector databases, [Qdrant](https://qdrant.tech/), and one of the popular embedding encoder and text transformer libraries, [SentenceTransformer](https://sbert.net/).

In [9]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# Create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [10]:
# Create collection to store the wine rating data
collection_name="top_wines"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

### Loading the data into the vector database

We will use the (vector) collection that we created above, to go over all the `notes` column of the wine dataset, and encode it into embedding vector, and store it in the vector database. The indexing of the data to allow quick retrieval is running in the background as we load it.

This step will take a few seconds (less than a minute on my laptop).

In [11]:
# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [12]:
console.print(qdrant.get_collection(collection_name=collection_name))

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=None,
    indexed_vectors_count=0,
    points_count=1347,
    segments_count=1,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=384,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=None,
            sharding_method=None,
            replication_factor=None,
            write_consistency_factor=None,
            read_fan_out_factor=None,
            on_disk_payload=None,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=None,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None,
        strict_mode_config=None
    ),
    payload_schema={}
)

## **R**etrieve sematically relevant data based on user's query

Once the data is loaded into the vector database and the indexing process is done, we can start using our simple RAG system.

In [13]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

### Encoding the user's query

We will use the same encoder that we used to encode the document data to encode the query of the user. 
This way we can search results based on semantic similarity. 

In [14]:
query_vector = encoder.encode(user_prompt).tolist()

### Search similar rows

We can now take the embedding encoding of the user's query and use it to find similar rows in the vector database.

In [15]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)

In [16]:
from rich.console import Console
from rich.text import Text
from rich.table import Table

table = Table(title="Retrieval Results", show_lines=True)

table.add_column("Name", style="#e0e0e0")
table.add_column("Region", style="bright_red")
table.add_column("Variety", style="green")
table.add_column("Rating", style="yellow")
table.add_column("Notes", style="#89ddff")
table.add_column("Score", style="#a6accd")

for hit in hits:
    table.add_row(
        hit.payload["name"],
        hit.payload["region"],
        hit.payload["variety"],
        str(hit.payload["rating"]),
        f'{hit.payload["notes"][:50]}...',
        f"{hit.score:.4f}"
    )

console.print(table)

                                                 Retrieval Results                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name                           ┃ Region           ┃ Variety  ┃ Rating ┃ Notes                          ┃ Score  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Catena Zapata Argentino        │ Argentina        │ Red Wine │ 98.0   │ "The single-vineyard 2004      │ 0.6378 │
│ Vineyard Malbec 2004           │                  │          │        │ Malbec Argentino Vineyar...    │        │
├────────────────────────────────┼──────────────────┼──────────┼────────┼────────────────────────────────┼────────┤
│ Bodega Colome Altura Maxima    │ Salta, Argentina │ Red Wine │ 96.0   │ Winemaker Thibaut Delmotte has │ 0.6180 │
│ Malbec 2012                    │                  │          │        │ crafted wines of di...         │        │
├────────────────────────────────┼──────────────────┼──────────┼────────┼────────────────────────────────┼────────┤
│ Catena Zapata Adrianna         │ Argentina        │ Red Wine │ 97.0   │ "The single-vineyard 2004      │ 0.6118 │
│ Vineyard Malbec 2004           │                  │          │        │ Malbec Adrianna Vineyard...    │        │
└────────────────────────────────┴──────────────────┴──────────┴────────┴────────────────────────────────┴────────┘

## **A**ugment the prompt to the LLM with retrieved data

In our simple example, we will simply take the top 3 results and use them as is in the prompt to the generation LLM.

## **G**enerate reply to the user's query

We will use Gemma3:4b from Ollama and run it locally on GTX 1050 Ti 4Gigs GPU

### First let's try without **R**etrieval

We can ask the LLM to recommend based only on the user prompt.

In [28]:
from ollama import chat
from rich.panel import Panel

response = chat(
    model="gemma3:4b",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Here is my wine recommendation:"}
    ],
    stream=False
)
styled_panel = Panel(
    response.message.content,
    title="Wine Recommendation without Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭───────────────────────────────────── Wine Recommendation without Retrieval ─────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│ **For a truly amazing Malbec experience from Argentina, I wholeheartedly recommend you try the Catena Zapata    │
│ Malbec – Alto Pago Vieras.**                                                                                    │
│                                                                                                                 │
│ Let me tell you why it's consistently rated as one of the best and why I think it’ll be amazing for you:        │
│                                                                                                                 │
│ *   **Estate-Grown Excellence:** The Vieiras vineyard is located in the Uco Valley, one of Argentina's premier  │
│ wine regions. Catena Zapata has meticulously cultivated this site for decades, focusing on unique clones and    │
│ meticulous vineyard management.                                                                                 │
│ *   **Distinct Flavor Profile:** This Malbec consistently delivers incredibly complex flavors. Expect notes of: │
│     *   **Dark Fruits:** Blackberry, plum, and cherry are prominent, but with a deep, almost jam-like           │
│ character.                                                                                                      │
│     *   **Spicy Undertones:** You'll find hints of black pepper, cloves, and cinnamon—a hallmark of the Uco     │
│ Valley’s high altitude and cool evenings.                                                                       │
│     *   **Vanilla & Oak:** The wine is aged in new French oak barrels, contributing subtle vanilla, toast, and  │
│ spice notes that beautifully complement the fruit.                                                              │
│     *   **Mineral Complexity:** A notable minerality adds a sophisticated dimension to the wine.                │
│ *   **Aging Potential:**  This is a wine that improves with age.  Even a younger bottle (3-5 years) will be     │
│ fantastic, but it has the potential to evolve beautifully for 10+ years in the cellar.                          │
│ *   **Reputation:** Catena Zapata is a household name in Argentina's wine scene. They’re pioneers of            │
│ single-vineyard Malbec and have earned critical acclaim worldwide.                                              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│ **Here’s a little more about it to help you decide:**                                                           │
│                                                                                                                 │
│ *   **Vintage:**  2019, 2020, and 2022 are particularly stellar vintages.  2019 is still fantastic, but the     │
│ more recent vintages have been exceptional.                                                                     │
│ *   **Price Range:** Expect to pay around $45 - $75 depending on the vintage and retailer.                      │
│ *   **Food Pairing:** This wine is a fantastic match for grilled meats (especially steak), lamb, roasted        │
│ vegetables, hard cheeses, or even rich mushroom dishes.                                                         │
│                                                       

In [36]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

console.print(search_results[0])

{
    'name': 'Catena Zapata Argentino Vineyard Malbec 2004',
    'region': 'Argentina',
    'variety': 'Red Wine',
    'rating': 98.0,
    'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably 
fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and 
blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing
a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional
achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard
of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his 
daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans 
of both architecture and wine in Mendoza. It is hard to believe, given the surge in popularity of Malbec in recent 
years, that Catena Zapata only began exporting Malbec to the United States in 1994."'
}

In [38]:
from ollama import chat
from rich.panel import Panel

response = chat(
    model="gemma3:4b",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ],
    stream=False
)
styled_panel = Panel(
    response.message.content,
    title="Wine Recommendation with Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭────────────────────────────────────── Wine Recommendation with Retrieval ───────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│ Okay, let's find you an *amazing* Malbec from Argentina! Based on the information I have, I’m going to          │
│ recommend the **Catena Zapata Argentino Vineyard Malbec 2004**.                                                 │
│                                                                                                                 │
│ Here’s why it’s a fantastic choice:                                                                             │
│                                                                                                                 │
│ *   **Exceptional Rating:** It consistently receives a 98.0 rating, indicating it’s a truly top-tier wine.      │
│ *   **Complex Flavors:** It boasts a remarkably complex flavor profile, including notes of wood smoke,          │
│ creosote, pepper, clove, black cherry, and blackberry – creating a truly layered tasting experience.            │
│ *   **Legacy & Production:** It comes from Catena Zapata, which is widely regarded as *the* benchmark for       │
│ Malbec in Argentina. They're pioneers and innovators in the region.                                             │
│ *   **Age Potential:**  Even now, at 20 years old, it is considered to be a wine with an extremely long life    │
│ ahead.                                                                                                          │
│                                                                                                                 │
│ **To help me narrow it down further and give you an even more tailored recommendation, could you tell me:**     │
│                                                                                                                 │
│ 1.  **What's your budget?** (e.g., Under $30, $30-$60, $60+, no limit!)                                         │
│ 2.  **What kind of food are you planning to pair it with?** (e.g., Steak, pasta, grilled vegetables, spicy      │
│ dishes?)                                                                                                        │
│ 3.  **What are you generally looking for in a wine?** (e.g., Fruity and bold, elegant and balanced, complex and │
│ aged?)                                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯